## Particle Swarm Optimization (PSO)
PSO는 SVM 매개변수 조정이나 퍼지 클러스터링에서 사용되며, 빠르고 안정적인 성능을 보임

In [ ]:
import pandas as pd
import numpy as np
from pyswarm import pso
from joblib import load

# 1. Surrogate 모델 불러오기 (pkl 파일)
surrogate_model = load('/data/ephemeral/home/Sojeong/level4-cv-finalproject-hackathon-cv-02-lv3/Sojeong/surrogate/autosklearn_model.pkl')

# # 최적 모델 가져오기
# models_with_weights = surrogate_model.get_models_with_weights()
# best_model = models_with_weights[0][1]  # 최적 모델만 추출 (파이프라인 포함)

# 2. 데이터셋 불러오기
df = pd.read_csv("/data/ephemeral/home/Sojeong/level4-cv-finalproject-hackathon-cv-02-lv3/Sojeong/surrogate/melb_split.csv")
data = df.drop(['Price', 'Split', 'Address', 'BuildingArea', 'YearBuilt'], axis=1)  # 제외한 데이터

# 범주형 데이터 원-핫 인코딩
# categorical_columns = ["Suburb", "Type", "Method", "SellerG", "Date", "CouncilArea", "Regionname"] # Data는 어떻게 하지 
# data = pd.get_dummies(data, columns=categorical_columns, drop_first=True)

# 3. 목표 함수 정의
def objective_function(x):
    """
    PSO의 목적 함수
    x: 최적화 대상 변수 리스트 (['Distance', 'Landsize'])
    return: 목표값과 예측값(Predicted) 간의 차이
    """
    target_price = 10000000  # 최적화 목표값
    
    # Distance와 Landsize 열을 업데이트
    data_copy = data.copy()
    data_copy["Distance"] = x[0]
    data_copy["Landsize"] = x[1]

    # Price 열 제거 (모델 학습 시 타겟 열)
    input_features = data_copy#.drop(columns=["Price"])

    # Surrogate 모델로 예측 수행
    predicted_prices = surrogate_model.predict(input_features)
    print("Predicted 값:", predicted_prices)

    # 모든 예측 값과 목표값의 평균 차이 반환
    return np.mean(np.abs(predicted_prices - target_price)) # y_hat - y(Price값이 아니라 우리가 원하는 목표값)

# 4. PSO 최적화
# 각 변수의 범위 설정
lb = [0, 0]  # 'Distance', 'Landsize' 최소값
ub = [38, 4583]  # 'Distance', 'Landsize' 최대값

# PSO 실행
best_x, best_f = pso(objective_function, lb, ub, swarmsize=30, maxiter=100, debug=True)

# 결과 출력
print("최적화된 특성 값 ('Distance', 'Landsize'):", best_x)
print("목표값과의 차이:", best_f)

No constraints given.
Predicted 값: [1019395.44726562  559725.25488281  572826.23046875 ...  619186.17773438
  655801.83105469  549007.03125   ]
Predicted 값: [1062570.47851562  569321.61425781  582592.73046875 ...  636753.02929688
  683266.04980469  583828.203125  ]
Predicted 값: [991185.16210938 411558.49707031 547815.95507812 ... 500727.51074219
 557866.890625   521714.60351562]
Predicted 값: [970339.51367188 526478.57226562 492621.78417969 ... 600591.19238281
 652966.3828125  537079.34960938]
Predicted 값: [1062199.08203125  569917.91601562  566385.6484375  ...  655553.51757812
  742611.58691406  647020.84570312]
Predicted 값: [1528235.51757812  805941.91210938  967086.55078125 ...  756900.390625
 1064011.52929688  920962.99609375]
Predicted 값: [1005154.43554688  528284.33789062  504831.01855469 ...  613682.27832031
  669840.6796875   563133.92773438]
Predicted 값: [1228859.57421875  699286.15527344  730606.40625    ...  713328.57421875
  807084.19824219  777828.06835938]
Predicted 값: [10

In [58]:
# 최적화된 결과를 Surrogate 모델로 확인
# Distance와 Landsize 열을 업데이트
data_copy1 = data.copy()
data_copy1["Distance"] = best_x[0]
data_copy1["Landsize"] = best_x[1]

# Price 열 제거 (모델 학습 시 타겟 열)
input_features1 = data_copy1#.drop(columns=["Price"])

# Surrogate 모델로 예측 수행
final_predicted_price = surrogate_model.predict(input_features1)
print(final_predicted_price.shape)    
print("최적화된 Predicted 값:", final_predicted_price)

(13580,)
최적화된 Predicted 값: [1788829.30078125  894908.484375   1109779.00390625 ...  955726.54492188
 1457456.4765625  1029908.5703125 ]


In [59]:
best_x

array([  10.        , 1486.99997359])